<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/01_musik_der_gro%C3%9Fst%C3%A4dte/musik_der_gro%C3%9Fst%C3%A4dte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Musik der Großstädte

Der Musikdienst lieferte Daten über die von Nutzern in Moskau und St. Petersburg abgehörten Tracks für einen bestimmten Zeitraum.
Es ist notwendig, Fragen zu beantworten und Hypthesen zu überprüfen:
- musik in zwei Städten — Moskau und St. Petersburg - hören Sie in verschiedenen Modi;
- die Listen der zehn beliebtesten Genres am Montagmorgen und am Freitagabend haben charakteristische Unterschiede;
- die Bewohner der beiden Städte bevorzugen verschiedene Musikgenres.

In [1]:
# Wir importieren die notwendigen Bibliotheken.
import pandas as pd

In [ ]:
# Wir laden Tabellen mit Daten.
df = pd.read_csv('/content/music_project.csv')

In [4]:
df = pd.read_csv('/content/music_project.csv')

In [7]:
# Wir werden die erhaltenen Daten studieren
data.info()
display(data.shape)
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


(65079, 7)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
